In [1]:
import matplotlib.pyplot as plt
import random
import imageio

In [2]:
class Person:
    def __init__(self, x, y, infected_prob, daily_info):
        self.x = x
        self.y = y
        
        # Tentukan apakah orang tersebut sehat atau
        # terinfeksi menggunakan peluang terinfeksi.
        temp = random.random()
        if temp <= infected_prob:
            self.is_healthy = False
            self.day_infected = 1
            daily_info['infected'][0] += 1
        else:
            self.is_healthy = True
            self.day_infected = 0
            daily_info['healthy'][0] += 1
        
        self.is_immuned = False
        
    def move(self, move_prob, area_limit):
        if random.random() <= move_prob:
            temp = random.random()
            if temp <= 0.25:
                self.x += 1
            elif temp <= 0.50:
                self.x -= 1
            elif temp <= 0.75:
                self.y += 1
            else:
                self.y -= 1
        
        # Handle koordinat yang melebihi batas.
        if self.x > area_limit['x']:
            self.x -= area_limit['x']
        elif self.x < 0:
            self.x += area_limit['x']
            
        if self.y > area_limit['y']:
            self.y -= area_limit['y']
        elif self.y < 0:
            self.y += area_limit['y']
        
        if not self.is_healthy:
            self.day_infected += 1
            
    def check_health(self, persons, daily_info, day, recovery_day):
        # Cek apakah terinfeksi virus, orang tersebut terinfeksi
        # jika satu koordinat dengan orang yang terinfeksi.
        if (self.is_healthy and not self.is_immuned):
            for person in persons:
                if (not person.is_healthy
                        and self.x == person.x 
                        and self.y == person.y):
                    self.is_healthy = False
                    self.day_infected += 1
                    daily_info['healthy'][day-1] -= 1
                    daily_info['infected'][day-1] += 1
                    break

        # Jika orang tersebut sudah terinfeksi, maka cek apakah
        # orang tersebut sudah sembuh. Dikatakan sembuh apabila
        # hari terinfeksi-nya sudah melewati hari penyembuhan.
        elif (not self.is_healthy and 
              self.day_infected > recovery_day):
            self.is_healthy = True
            self.is_immuned = True
            daily_info['healthy'][day-1] += 1
            daily_info['infected'][day-1] -= 1
            daily_info['recovered'][day-1] += 1

In [3]:
def create_map(persons, day, name):
    # Buat peta penyebaran covid-19 pada hari yang 
    # telah ditentukan dan save gambar tersebut.
    
    # Buat list koordinat untuk orang yang sehat, sakit, dan sembuh.
    healthy_person = {'x' : [], 'y' : []}
    infected_person = {'x' : [], 'y' : []}
    recovered_person = {'x' : [], 'y' : []}
    for person in persons:
        if person.is_immuned:
            recovered_person['x'].append(person.x)
            recovered_person['y'].append(person.y)
        elif person.is_healthy:
            healthy_person['x'].append(person.x)
            healthy_person['y'].append(person.y)
        else:
            infected_person['x'].append(person.x)
            infected_person['y'].append(person.y)

    plt.plot(healthy_person['x'], healthy_person['y'],
             'o', color = '#00bdaa', label='Sehat')
    plt.plot(infected_person['x'], infected_person['y'],
             'o', color="#d7385e", label='Sakit')
    plt.plot(recovered_person['x'], recovered_person['y'],
             'o', color='#f0a500', label='Sembuh')

    plt.xticks([])
    plt.yticks([])

    plt.title(f'Peta Penyebaran Covid-19 di Wilayah X Hari ke-{day}')
    plt.legend(loc='upper left')
    
    plt.savefig(f'covid/img/{name}.png')
    plt.close()

In [4]:
def create_infographic(daily_info, day, name):
    # Buat infografik hari terhadap orang yang
    # sehat, sakit dan yang sembuh
    day_list = [i+1 for i in range(day)]

    plt.plot(day_list, daily_info['healthy'],
             color='#00bdaa', label='Sehat')
    plt.plot(day_list, daily_info['infected'],
             color='#d7385e', label='Sakit')
    plt.plot(day_list, daily_info['recovered'],
             color='#f0a500', label='Sembuh')

    plt.xlabel('Hari ke-')
    plt.ylabel('Jumlah Penduduk')

    plt.title(f'Grafik Kasus Covid-19 di Wilayah X')
    plt.legend(loc='upper left')

    plt.savefig(f'covid/img/{name}.png')
    plt.close()

In [5]:
# Inisialisasi semua variabel.
area_limit = {'x' : 20, 'y' : 20}

individu_amount = 200
infected_prob = 0.05
move_prob = 0.8
recovery_day = 10

daily_info = {
    'healthy' : [0],
    'infected' : [0],
    'recovered' : [0]
}

In [6]:
# Main Program
persons = []

day = 1
for i in range(individu_amount):
    x = int(random.random() * area_limit['x'])
    y = int(random.random() * area_limit['y'])
    person = Person(x, y, infected_prob, daily_info)
    persons.append(person)
    
create_map(persons, day, f'map-day-{day}')
create_infographic(daily_info, day, f'info-day-{day}')
    
day += 1
while daily_info['infected'][day-2] > 0:
    daily_info['healthy'].append(daily_info['healthy'][day-2])
    daily_info['infected'].append(daily_info['infected'][day-2])
    daily_info['recovered'].append(daily_info['recovered'][day-2])
    
    for person in persons:
        person.move(move_prob, area_limit)
        
    for person in persons:
        person.check_health(persons, daily_info, day, recovery_day)
    
    create_map(persons, day, f'map-day-{day}')
    create_infographic(daily_info, day, f'info-day-{day}')
    
    day += 1

In [7]:
print(f'Hari terakhir adalah hari ke-{day-1}')

Hari terakhir adalah hari ke-50


In [8]:
# Buat animasinya.
map_img = []
info_img = []

for i in range(day-1):
    map_img.append(imageio.imread(f'covid/img/map-day-{i+1}.png'))
    info_img.append(imageio.imread(f'covid/img/info-day-{i+1}.png'))
    
imageio.mimsave('covid/map.gif', map_img)
imageio.mimsave('covid/info.gif', info_img)